In [1]:
from nsfd.uvp import *
from nsfd.boundary import *
from nsfd.typing import BCondition


imax = 128
jmax = 128
xlength = 1.0
ylength = 1.0
delt = 0.02
tau = 0.5
eps = 0.001
omg = 1.7
gamma = 0.9
itermax = 100
GX = 0.0
GY = 0.0
Re = 1000
UI = 0.0
VI = 0.0
PI = 0.0
wW = BCondition.NO_SLIP
wE = BCondition.NO_SLIP
wS = BCondition.NO_SLIP
wN = BCondition.NONE

u_lid = 1.0

delx = xlength / imax
dely = ylength / jmax

u, v, p = init_uvp(imax, jmax, UI, VI, PI)

# time step
set_b_cond(u, v, imax, jmax, wW, wE, wN, wS)
u[:, jmax + 1] = 2.0 * u_lid - u[:, jmax]
v[:, jmax + 1] = 0

F = np.zeros_like(u)
G = np.zeros_like(u)
RHS = np.zeros_like(u)
res = np.zeros_like(RHS)


In [2]:
comp_fg(u, v, F, G, imax, jmax, delt, delx, dely, GX, GY, gamma, Re)

In [3]:
# F[imax,jmax] = F[imax-1,jmax]

In [4]:
1/delt*(F[1:,:] - F[-1:,:])/delx

array([[    0.   ,     0.   ,     0.   , ...,     0.   ,  4194.304,
            0.   ],
       [    0.   ,     0.   ,     0.   , ...,     0.   ,  4194.304,
            0.   ],
       [    0.   ,     0.   ,     0.   , ...,     0.   ,  4194.304,
            0.   ],
       ...,
       [    0.   ,     0.   ,     0.   , ...,     0.   ,  4194.304,
            0.   ],
       [   -0.   ,     0.   ,     0.   , ...,     0.   ,     0.   ,
        12800.   ],
       [    0.   ,     0.   ,     0.   , ...,     0.   ,     0.   ,
            0.   ]])

In [5]:
comp_rhs(F, G, RHS, imax, jmax, delt, delx, dely)

In [6]:
RHS[:,:-2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
it, res_norm = poisson(p, RHS, imax, jmax, delx, dely, eps, itermax, omg, res)

In [8]:
res_norm

21.861121806567553

In [15]:
RHS[:,-2]

array([    0.   ,  4194.304,     0.   ,     0.   ,     0.   ,     0.   ,
           0.   ,     0.   ,     0.   ,     0.   ,     0.   ,     0.   ,
           0.   ,     0.   ,     0.   ,     0.   ,     0.   ,     0.   ,
           0.   ,     0.   ,     0.   ,     0.   ,     0.   ,     0.   ,
           0.   ,     0.   ,     0.   ,     0.   ,     0.   ,     0.   ,
           0.   ,     0.   ,     0.   ,     0.   ,     0.   ,     0.   ,
           0.   ,     0.   ,     0.   ,     0.   ,     0.   ,     0.   ,
           0.   ,     0.   ,     0.   ,     0.   ,     0.   ,     0.   ,
           0.   ,     0.   ,     0.   ,     0.   ,     0.   ,     0.   ,
           0.   ,     0.   ,     0.   ,     0.   ,     0.   ,     0.   ,
           0.   ,     0.   ,     0.   ,     0.   ,     0.   ,     0.   ,
           0.   ,     0.   ,     0.   ,     0.   ,     0.   ,     0.   ,
           0.   ,     0.   ,     0.   ,     0.   ,     0.   ,     0.   ,
           0.   ,     0.   ,     0.   ,     0.   , 

In [ ]:
adap_uv(u, v, F, G, p, imax, jmax, delt, delx, dely)

In [ ]:
# time step
set_b_cond(u, v, imax, jmax, wW, wE, wN, wS)
u[:, jmax + 1] = 2.0 * u_lid - u[:, jmax]
v[:, jmax + 1] = 0

F = np.empty_like(u)
G = np.empty_like(u)
RHS = np.empty_like(u)
res = np.empty_like(RHS)

comp_fg(u, v, F, G, imax, jmax, delt, delx, dely, GX, GY, gamma, Re)
comp_rhs(F, G, RHS, imax, jmax, delt, delx, dely)
it, res_norm = poisson(p, RHS, imax, jmax, delx, dely, eps, itermax, omg, res)
adap_uv(u, v, F, G, p, imax, jmax, delt, delx, dely)